# PCCP.C1 Plots

[Click here](https://www.arm.gov/capabilities/vaps/pccp) for more information about this vap.

In [ ]:
%matplotlib widget
import ipywidgets as widgets

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd
import os
from datetime import datetime

import act
import xarray as xr

# Data archive directory
DATA_DIR = r'/data/archive/'

# Datastream info
DATASTREAM_NAME = 'pccp'
DATA_LEVEL = 'c1'
LOCATIONS = [{'end_date': '2020-03-03', 'facility': 'E43', 'site': 'sgp', 'start_date': '2017-09-01'}, {'end_date': '2019-12-01', 'facility': 'E44', 'site': 'sgp', 'start_date': '2017-09-01'}, {'end_date': '2019-10-30', 'facility': 'E45', 'site': 'sgp', 'start_date': '2017-09-01'}]

## Define site, facility, and date ranges

In [ ]:
print("The following locations and date ranges are available for this VAP:")
display(pd.DataFrame(LOCATIONS, columns=['site', 'facility', 'start_date', 'end_date']))

In [ ]:
# Define site, facility, and date range below:
site_facility = ( 'sgp', 'E45' )

# Date range. Format: YYYY-MM-DD
date_start = '2019-10-24'
date_end = '2019-10-25'

## Load data files
Loads data files from /data/archive/

In [ ]:
# Compile list of files
site, facility = site_facility
d_date_start = datetime.strptime(date_start, '%Y-%m-%d')
d_date_end = datetime.strptime(date_end, '%Y-%m-%d')
dir_path = os.path.join(DATA_DIR + site, site + DATASTREAM_NAME + facility + r'.' + DATA_LEVEL )
files_list  = []
for f in os.listdir(dir_path):
    file_date_str = f.split(r'.')[2]
    try:
        file_date = datetime.strptime(file_date_str, '%Y%m%d')
    except Exception:
        continue 
    if d_date_start <= file_date and d_date_end > file_date:
        files_list.append(os.path.join(dir_path, f))

# Load files as a single dataset
ds = act.io.armfiles.read_netcdf(files_list)
ds.clean.cleanup()
print(f'{len(files_list)} files loaded')
ds


## Point Cloud of Cloud of Points

In [ ]:
# this variable represents the index of the data point that is being shown
print(f"Available time values: {ds.time.dt.strftime(r'%Y-%m-%d %H:%M:%S').values[0]} -- {ds.time.dt.strftime(r'%Y-%m-%d %H:%M:%S').values[-1]}")


In [ ]:
# Enter timestamp to plot (format: YYYY-MM-DD hh:mm:ss)
display_time = '2019-10-24 15:00:00'

# list available time stamps
display_dt = datetime.strptime(display_time, r'%Y-%m-%d %H:%M:%S')
available_times = np.array([datetime.combine(d,t) for d, t in zip(ds.time.dt.date.values,ds.time.dt.time.values)])
# get closest time 
time_index = np.argmin(np.abs(available_times - display_dt))


In [ ]:
x_relative_var = ds.variables['x_relative'][time_index]
y_relative_var = ds.variables['y_relative'][time_index]
z_relative_var = ds.variables['z_relative'][time_index]

# # Filter out values that exceed 50 km
ind_nonzero = tuple(np.nonzero((np.abs(x_relative_var) < 50000)))
print(len(x_relative_var[0]),' cloud points are extracted')


In [ ]:
x_slice = np.array(x_relative_var[ind_nonzero])/1000 # convert to km 
y_slice = np.array(y_relative_var[ind_nonzero])/1000
z_slice = np.array(z_relative_var[ind_nonzero])/1000

###2D plot of x,y variables in subplot(2,2,si)
def plot2D(x,y,fig,si,xlabel,ylabel):
    ax = fig.add_subplot(2,2,si)
    ax.scatter(x, y, s=4, marker='o', c= 'gray')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.xaxis.labelpad = 5
    ax.yaxis.labelpad = 5

###3D plot of x,y,z variables in subplot(2,2,si)
def plot3D(x,y,z,fig,si):
    #check if data point count is sufficient for display
    if (len(x)>10):
        ax = fig.add_subplot(1,1,si, projection='3d')
        x = [x[0:len(x)]]
        y = [y[0:len(y)]]
        z = [z[0:len(z)]]
        x1 = int(min(min(x)))
        x2 = int(max(max(x)))
        y1 = int(min(min(y)))
        y2 = int(max(max(y)))
        z2 = int(max(max(z)))
      
        ax.scatter(x, y, z, c='gray', marker='o')
        # ax.xaxis.set_ticks(np.arange(x1,x2,int((x2-x1+2)/4)+0.5))
        # ax.yaxis.set_ticks(np.arange(y1,y2,int((y2-y1+2)/4)+0.5))
        ax.view_init(elev=15, azim=-70)
        ax.set_xlabel('X [km] ')
        ax.set_ylabel('Y [km] ')
        ax.xaxis.labelpad = 15
        ax.yaxis.labelpad = 15
        ax.zaxis.set_ticks(np.arange(0,int(z2+1),.5))
        ax.set_zlabel('Z [km] ')
        
fig = plt.figure(figsize=(15,10))
plot2D(x_slice,z_slice,fig,1,'direction eastward [km]','altitude above the ground [km]')
plot2D(y_slice,z_slice,fig,2,'direction northward [km]','altitude above the ground [km]')
plot2D(x_slice,y_slice,fig,3,'direction eastward [km]','direction northward [km]')
fig = plt.figure(figsize=(15,10))
plot3D(x_slice,y_slice,z_slice,fig,1)